In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
pd.set_option('display.max_columns', None)

## Lectura de datos

In [ ]:
df = pd.read_parquet("top_5_delay.parquet")

## Análisis de las causas de retraso

Vamos a explorar las causas de retraso en estos aeropuertos. En primer lugar, analizaremos el impacto en el tiempo de retraso de cada una de estas causas. Después examinaremos el número de vuelos al que impactan cada una de estas causas

In [ ]:
airports = df.groupby(["ORIGIN_AIRPORT","ORIGIN_CITY"]).mean()
airports["FLIGHTS"] = df.groupby(["ORIGIN_AIRPORT","ORIGIN_CITY"]).size()
airports["DELAYED_FLIGHTS"] = df[df["ARRIVAL_DELAY"]>0].groupby(["ORIGIN_AIRPORT","ORIGIN_CITY"]).size()
airports["DELAYED_PERCENTAGE"] = airports["DELAYED_FLIGHTS"]/airports["FLIGHTS"]
airports = airports.reset_index()
airports

Añadimos variables recuento

In [ ]:
air_sys_delay = []
security_delay = []
airline_delay = []
late_aircrat_delay = []
weather_delay = []
other_delay = []

origin_airports = list(airports['ORIGIN_AIRPORT'].unique())
for origin in origin_airports:
    #print(origin)
    data = df[df['ORIGIN_AIRPORT']==origin]
    air_sys_delay.append(np.count_nonzero(data['AIR_SYSTEM_DELAY']))
    security_delay.append(np.count_nonzero(data['SECURITY_DELAY']))
    airline_delay.append(np.count_nonzero(data['AIRLINE_DELAY']))
    late_aircrat_delay.append(np.count_nonzero(data['LATE_AIRCRAFT_DELAY'])) 
    weather_delay.append(np.count_nonzero(data['WEATHER_DELAY']))
    other_delay.append(np.count_nonzero(data['OTHER_DELAY']))
    
airports["COUNT_AIR_SYSTEM_DELAY"] = air_sys_delay
airports["COUNT_SECURITY_DELAY"] = security_delay
airports["COUNT_AIRLINE_DELAY"] = airline_delay
airports["COUNT_LATE_AIRCRAFT_DELAY"] = late_aircrat_delay
airports["COUNT_WEATHER_DELAY"] = weather_delay
airports["COUNT_OTHER_DELAY"] = other_delay
airports

In [ ]:
fig = make_subplots(rows=2, cols=5, subplot_titles=list(airports["ORIGIN_AIRPORT"].values)*2,
                    specs=[[{"type": "pie"}, {"type": "pie"}, {"type": "pie"}, {"type": "pie"}, {"type": "pie"}],
                           [{"type": "pie"}, {"type": "pie"}, {"type": "pie"}, {"type": "pie"}, {"type": "pie"}]])

delay_labels = ["AIR_SYSTEM_DELAY","SECURITY_DELAY","AIRLINE_DELAY","LATE_AIRCRAFT_DELAY","WEATHER_DELAY","OTHER_DELAY"]
count_labels = ["COUNT_AIR_SYSTEM_DELAY","COUNT_SECURITY_DELAY","COUNT_AIRLINE_DELAY","COUNT_LATE_AIRCRAFT_DELAY","COUNT_WEATHER_DELAY","COUNT_OTHER_DELAY"]

for i in range(5):
    values1 = airports[delay_labels].iloc[i]
    fig.add_trace(go.Pie(labels=delay_labels, values=values1, direction ='clockwise', marker_colors=px.colors.qualitative.Vivid, hole=.3),
              row=1, col=i+1)
    values2 = airports[count_labels].iloc[i]
    fig.add_trace(go.Pie(labels=delay_labels, values=values2, direction ='clockwise', marker_colors=px.colors.qualitative.Vivid, hole=.3),
              row=2, col=i+1)

fig.update_layout(title_text="Average Delay Distribution by Airport<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>Percentage of flights affected by each cause per airport", 
                  
                  legend_title="Delay Cause", template="plotly_dark", height=750,
                    legend=dict(orientation="h", y=0.63, x =0.015))
fig.update_annotations(yshift=-119)

fig.show()

Estos gráficos corresponden con el tiempo medio de retraso provocado por cada una de las causas en cada uno de los aeropuertos. Por ejemplo, vemos que en el caso de Atlanta la causa que provoca retrasos más largos es la arelína, que provoca un retraso medio de 3.4 minutos [...]

Por otro lado, la segunda fila representa el % de vuelos que se ven afectados por cada causa en cada uno de los aeropuertos, independientemente del tiempo de retraso. Siguendo con el caso de Atlanta, en esta ocasión vemos que la mayoría de vuelos sufren un retraso por causa no desconicida. 

Si juntamos la información de ambas gráficas vemos que en Atlanta la mayoría de vuelos se retrasan por causa desconocida, aunque el tiempo de retraso asociado a esta causa suele ser pequeño, en torno a 1 minuto. Así mismo observamos que la causa que mayores retrasos produce, retraso por culpa de la aerolína, afecta únicamente a un 21.9% de los vuelos